### Prompt Chaining

Prompt chaining is a powerful pattern that breaks down complex AI tasks into a sequence of smaller, more focused steps. Each step in the chain processes the output from the previous step, allowing for better control, validation, and reliability.


In [1]:
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field
from openai import OpenAI
import os
import logging

In [2]:
# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4o"

### Calendar Assistant Example

Our calendar assistant demonstrates a 3-step prompt chain with validation:

![Alt Text](./images/Prompt_Chaining.png)



#### Step 1: Define the data models for each stage


##### Step 1.1: Extract & Validate

- Determines if the input is actually a calendar request
- Provides a confidence score
- Acts as an initial filter to prevent processing invalid requests



In [4]:
class EventExtraction(BaseModel):
    """First LLM call: Extract basic event information"""

    description: str = Field(description="Raw description of the event")
    is_calendar_event: bool = Field(
        description="Whether this text describes a calendar event"
    )
    confidence_score: float = Field(description="Confidence score between 0 and 1")


##### Step 1.2: Parse Details

- Extracts specific calendar information
- Structures the data (date, time, participants, etc.)
- Converts natural language to structured data



In [5]:
class EventDetails(BaseModel):
    """Second LLM call: Parse specific event details"""

    name: str = Field(description="Name of the event")
    date: str = Field(
        description="Date and time of the event. Use ISO 8601 to format this value."
    )
    duration_minutes: int = Field(description="Expected duration in minutes")
    participants: list[str] = Field(description="List of participants")


##### Step 1.3: Generate Confirmation

- Creates a user-friendly confirmation message
- Optionally generates calendar links
- Provides the final user response

In [6]:
class EventConfirmation(BaseModel):
    """Third LLM call: Generate confirmation message"""

    confirmation_message: str = Field(
        description="Natural language confirmation message"
    )
    calendar_link: Optional[str] = Field(
        description="Generated calendar link if applicable"
    )

#### Step 2: Define the functions


In [7]:
def extract_event_info(user_input: str) -> EventExtraction:
    """First LLM call to determine if input is a calendar event"""
    logger.info("Starting event extraction analysis")
    logger.debug(f"Input text: {user_input}")

    today = datetime.now()
    date_context = f"Today is {today.strftime('%A, %B %d, %Y')}."

    response = client.responses.parse(
    model="gpt-4o",
    input=user_input,
    instructions=f"{date_context} Analyze if the text describes a calendar event.",
    text_format=EventExtraction,
    )
    result = response.output[0].content[0].parsed
    
    logger.info(
        f"Extraction complete - Is calendar event: {result.is_calendar_event}, Confidence: {result.confidence_score:.2f}"
    )
    return result

In [8]:
def parse_event_details(description: str) -> EventDetails:
    """Second LLM call to extract specific event details"""
    logger.info("Starting event details parsing")

    today = datetime.now()
    date_context = f"Today is {today.strftime('%A, %B %d, %Y')}."

    response = client.responses.parse(
    model="gpt-4o",
    input=description,
    instructions=f"{date_context} Extract detailed event information. When dates reference 'next Tuesday' or similar relative dates, use this current date as reference.",
    text_format=EventDetails,
    )
    result = response.output[0].content[0].parsed
    
    logger.info(
        f"Parsed event details - Name: {result.name}, Date: {result.date}, Duration: {result.duration_minutes}min"
    )
    logger.debug(f"Participants: {', '.join(result.participants)}")
    return result

In [9]:
def generate_confirmation(event_details: EventDetails) -> EventConfirmation:
    """Third LLM call to generate a confirmation message"""
    logger.info("Generating confirmation message")

    response = client.responses.parse(
    model="gpt-4o",
    input=str(event_details.model_dump()),
    instructions="Generate a natural confirmation message for the event. Sign of with your name; Susie",
    text_format=EventConfirmation,
    )
    
    result = response.output[0].content[0].parsed

    logger.info("Confirmation message generated successfully")
    return result

#### Step 3: Chain the functions together


In [10]:
def process_calendar_request(user_input: str) -> Optional[EventConfirmation]:
    """Main function implementing the prompt chain with gate check"""
    logger.info("Processing calendar request")
    logger.debug(f"Raw input: {user_input}")

    # First LLM call: Extract basic info
    initial_extraction = extract_event_info(user_input)

    # Gate check: Verify if it's a calendar event with sufficient confidence
    if (
        not initial_extraction.is_calendar_event
        or initial_extraction.confidence_score < 0.7
    ):
        logger.warning(
            f"Gate check failed - is_calendar_event: {initial_extraction.is_calendar_event}, confidence: {initial_extraction.confidence_score:.2f}"
        )
        return None

    logger.info("Gate check passed, proceeding with event processing")

    # Second LLM call: Get detailed event information
    event_details = parse_event_details(initial_extraction.description)

    # Third LLM call: Generate confirmation
    confirmation = generate_confirmation(event_details)

    logger.info("Calendar request processing completed successfully")
    return confirmation

#### Step 4: Test the chain with a valid input

In [11]:
user_input = "Let's schedule a 1h team meeting next Tuesday at 2pm with Alice and Bob to discuss the project roadmap."

In [13]:
result = process_calendar_request(user_input)

2025-05-05 23:10:58 - INFO - Processing calendar request
2025-05-05 23:10:58 - INFO - Starting event extraction analysis
2025-05-05 23:10:59 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 23:10:59 - INFO - Extraction complete - Is calendar event: True, Confidence: 0.95
2025-05-05 23:10:59 - INFO - Gate check passed, proceeding with event processing
2025-05-05 23:10:59 - INFO - Starting event details parsing
2025-05-05 23:11:01 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 23:11:01 - INFO - Parsed event details - Name: Team Meeting, Date: 2025-05-13T14:00:00, Duration: 60min
2025-05-05 23:11:01 - INFO - Generating confirmation message
2025-05-05 23:11:03 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 23:11:03 - INFO - Confirmation message generated successfully
2025-05-05 23:11:03 - INFO - Calendar request processing completed successfully


In [14]:
if result:
    print(f"Confirmation: {result.confirmation_message}")
    if result.calendar_link:
        print(f"Calendar Link: {result.calendar_link}")
else:
    print("This doesn't appear to be a calendar event request.")

Confirmation: Hello Alice and Bob,

This is a confirmation for our upcoming Team Meeting scheduled for May 13, 2025, at 2:00 PM. The meeting will last approximately 60 minutes.

Looking forward to seeing you both there!

Best,
Susie



#### Step 5: Test the chain with an invalid input


In [15]:
user_input = "Can you send an email to Alice and Bob to discuss the project roadmap?"

In [16]:
result = process_calendar_request(user_input)


2025-05-05 23:11:08 - INFO - Processing calendar request
2025-05-05 23:11:08 - INFO - Starting event extraction analysis
2025-05-05 23:11:09 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-05-05 23:11:09 - INFO - Extraction complete - Is calendar event: False, Confidence: 0.90
2025-05-05 23:11:09 - WARNING - Gate check failed - is_calendar_event: False, confidence: 0.90


In [17]:
if result:
    print(f"Confirmation: {result.confirmation_message}")
    if result.calendar_link:
        print(f"Calendar Link: {result.calendar_link}")
else:
    print("This doesn't appear to be a calendar event request.")

This doesn't appear to be a calendar event request.
